In [1]:
import sys
print(sys.path)
sys.path.append("../KE/")
print(sys.path)

['/Users/marcin/Documents/VIMSS/ontology/KG-Hub/KG-Microbe/kg-microbe/notebooks', '/Users/marcin/.pyenv/versions/3.10.11/lib/python310.zip', '/Users/marcin/.pyenv/versions/3.10.11/lib/python3.10', '/Users/marcin/.pyenv/versions/3.10.11/lib/python3.10/lib-dynload', '', '/Users/marcin/Documents/VIMSS/ontology/KG-Hub/KG-Microbe/kg-microbe/venv/lib/python3.10/site-packages']
['/Users/marcin/Documents/VIMSS/ontology/KG-Hub/KG-Microbe/kg-microbe/notebooks', '/Users/marcin/.pyenv/versions/3.10.11/lib/python310.zip', '/Users/marcin/.pyenv/versions/3.10.11/lib/python3.10', '/Users/marcin/.pyenv/versions/3.10.11/lib/python3.10/lib-dynload', '', '/Users/marcin/Documents/VIMSS/ontology/KG-Hub/KG-Microbe/kg-microbe/venv/lib/python3.10/site-packages', '../KE/']


In [3]:
!pip install category_encoders


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.2 MB/s eta 0:00:00a 0:00:01
  Using cached statsmodels-0.14.0-cp310-cp310-macosx_11_0_arm64.whl (9.4 MB)
  Using cached patsy-0.5.3-py2.py3-none-any.whl (233 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool, cv

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

from KE.classifier.prepare import split_dataset
from KE.dataclean import tables

In [3]:
data = pd.read_csv("../data/merged/merged-kg_edges.tsv", header=0, sep="\t")

/var/folders/hx/svkm0dlj0bd9689lm21_7fyw0000gs/T/ipykernel_18758/2785372968.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../data/merged/merged-kg_edges.tsv", header=0, sep="\t")


In [4]:
data

,id,subject,predicate,object,relation,provided_by,knowledge_source,meta,primary_knowledge_source
0,urn:uuid:b6cd7c15-230e-43d1-ba8a-b2a8b1733cbe,NCBITaxon:1,biolink:subclass_of,BFO:0000040,rdfs:subClassOf,NaN,Graph,NaN,NaN
1,urn:uuid:88f29617-6c27-4b5d-8826-8e43248ddad9,NCBITaxon:10,biolink:subclass_of,NCBITaxon:1706371,rdfs:subClassOf,NaN,Graph,NaN,NaN
2,urn:uuid:43701742-c66a-4b28-87c5-236add5ab405,NCBITaxon:100,biolink:subclass_of,NCBITaxon:99,rdfs:subClassOf,NaN,Graph,NaN,NaN
3,urn:uuid:245e5bc2-090d-43af-ac7d-843903a9dd52,NCBITaxon:100,biolink:occurs_in,mediadive.medium:7,BAO:0002924,NaN,Graph,NaN,bacdive:17385
4,urn:uuid:50073882-f09b-418d-bb94-5832b7ce1af4,NCBITaxon:100,biolink:capable_of,traits.pathways:aerobic_chemo_heterotrophy,RO:0002215,NaN,Graph,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1469025,urn:uuid:1b21e8fd-6c66-4045-a252-026bac0e9e9d,NCBITaxon:147448,biolink:has_phenotype,traits.cell_shape_enum:coccus,RO:0002200,NaN,Graph,NaN,NaN
1469026,urn:uuid:567ad208-c607-40bd-8ec6-d2e73d753c81,NCBITaxon:147448,biolink:capable_of,ECOCORE:00000177,RO:0002215,NaN,Graph,NaN,NaN
1469027,urn:uuid:6981ca41-abda-4872-a5fe-1db001032db7,NCBITaxon:1240726,biolink:has_phenotype,traits.cell_shape_enum:bacillus,RO:0002200,NaN,Graph,NaN,NaN
1469028,urn:uuid:0a2c871e-5678-4fff-b2da-f9e22e709f36,NCBITaxon:1240783,biolink:has_phenotype,traits.cell_shape_enum:bacillus,RO:0002200,NaN,Graph,NaN,NaN


In [5]:
data_pairs = data[['subject','object']].drop_duplicates()
data_pairs

,subject,object
0,NCBITaxon:1,BFO:0000040
1,NCBITaxon:10,NCBITaxon:1706371
2,NCBITaxon:100,NCBITaxon:99
3,NCBITaxon:100,mediadive.medium:7
4,NCBITaxon:100,traits.pathways:aerobic_chemo_heterotrophy
...,...,...
1469025,NCBITaxon:147448,traits.cell_shape_enum:coccus
1469026,NCBITaxon:147448,ECOCORE:00000177
1469027,NCBITaxon:1240726,traits.cell_shape_enum:bacillus
1469028,NCBITaxon:1240783,traits.cell_shape_enum:bacillus


In [13]:
# Subset the DataFrame based on the substring in subject
data_pairs_clean = data_pairs[data_pairs['subject'].str.contains('NCBITaxon:')]
# Subset the DataFrame based on the substring in object
data_pairs_clean = data_pairs_clean[data_pairs_clean['object'].str.contains('mediadive.medium:')]
data_pairs_clean.to_csv("NCBITaxon_to_medium.tsv", sep="\t", header=True, index=False)
data_pairs_clean.shape

(8597, 2)

In [7]:
#add closure

data_pairs_chem = data_pairs[data_pairs['subject'].str.contains('NCBITaxon:')]
data_pairs_chem = data_pairs_chem[data_pairs_chem['object'].str.contains('CHEBI:')]
data_pairs_chem.shape

(37474, 2)

In [8]:
#add closure

data_pairs_go = data_pairs[data_pairs['subject'].str.contains('NCBITaxon:')]
data_pairs_go = data_pairs_go[data_pairs_go['object'].str.contains('GO:')]
data_pairs_go.shape

(2425, 2)

In [9]:
data_pairs_rest_all = data_pairs[data_pairs['subject'].str.contains('NCBITaxon:')]
data_pairs_rest = data_pairs_rest_all[data_pairs_rest_all['object'].str.contains('carbon_substrate:')]
data_pairs_rest2 = data_pairs_rest_all[data_pairs_rest_all['object'].str.contains('pathways:')]
data_pairs_rest = pd.concat([data_pairs_rest, data_pairs_rest2], ignore_index=True)
data_pairs_rest2 = data_pairs_rest_all[data_pairs_rest_all['object'].str.contains('cell_shape_enum:')]
data_pairs_rest = pd.concat([data_pairs_rest, data_pairs_rest2], ignore_index=True)
data_pairs_rest2 = data_pairs_rest_all[data_pairs_rest_all['object'].str.contains('data_source:')]
data_pairs_rest = pd.concat([data_pairs_rest, data_pairs_rest2], ignore_index=True)
data_pairs_rest.shape

(31692, 2)

In [62]:
data_df_pairs = pd.concat([data_pairs_chem, data_pairs_go], ignore_index=True)
data_df_pairs = pd.concat([data_df_pairs, data_pairs_rest], ignore_index=True)

data_df_pairs['Value'] = 1

# Step 2: Pivot the old DataFrame to form the new DataFrame structure
data_df = data_df_pairs.pivot(index='subject', columns='object', values='Value')

# Step 3: Fill NaN values with 0 to indicate no relationship
data_df = data_df.fillna(0)

# Optionally, convert the filled NaN values to integers if they were floats after pivot
data_df = data_df.astype(int)
data_df

object,CHEBI:132943,CHEBI:133748,CHEBI:14321,CHEBI:15361,CHEBI:15428,CHEBI:15792,CHEBI:15963,CHEBI:16015,CHEBI:16133,CHEBI:16193,...,traits.pathways:thiocyanate_oxidation,traits.pathways:thiocyanate_reduction,traits.pathways:thiosulfate_disproportionation,traits.pathways:thiosulfate_oxidation,traits.pathways:thiosulfate_oxidation_dark,traits.pathways:thiosulfate_reduction,traits.pathways:trithionate_oxidation,traits.pathways:uraninite_oxidation,traits.pathways:uranium_oxidation,traits.pathways:uranyl_reduction
subject,,,,,,,,,,,,,,,,,,,,,
NCBITaxon:100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
NCBITaxon:1000560,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NCBITaxon:1000561,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NCBITaxon:1000565,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NCBITaxon:1000567,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NCBITaxon:999702,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NCBITaxon:999891,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NCBITaxon:999892,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
subject_object_mapping = data_pairs_clean.set_index('subject')['object'].to_dict()

# Use the map function to create a new column in 'new_dataframe' from the 'subject' indices
data_df['medium'] = data_df.index.map(subject_object_mapping)


In [64]:
data_df['medium'].isna().sum()

21003

In [65]:
data_df = data_df[data_df['medium'].notna()]
data_df

object,CHEBI:132943,CHEBI:133748,CHEBI:14321,CHEBI:15361,CHEBI:15428,CHEBI:15792,CHEBI:15963,CHEBI:16015,CHEBI:16133,CHEBI:16193,...,traits.pathways:thiocyanate_reduction,traits.pathways:thiosulfate_disproportionation,traits.pathways:thiosulfate_oxidation,traits.pathways:thiosulfate_oxidation_dark,traits.pathways:thiosulfate_reduction,traits.pathways:trithionate_oxidation,traits.pathways:uraninite_oxidation,traits.pathways:uranium_oxidation,traits.pathways:uranyl_reduction,medium
subject,,,,,,,,,,,,,,,,,,,,,
NCBITaxon:100,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,mediadive.medium:7
NCBITaxon:1001240,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,mediadive.medium:92
NCBITaxon:100133,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,mediadive.medium:860
NCBITaxon:1003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,mediadive.medium:357
NCBITaxon:1004261,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,mediadive.medium:1081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NCBITaxon:999549,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,mediadive.medium:514
NCBITaxon:999550,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,mediadive.medium:514
NCBITaxon:999611,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,mediadive.medium:514


In [66]:
#data_pairs_clean = tables.rows_ad_cols_quant_filter(data_pairs)
#require at least 10 examples per class
data_df_clean = data_df.groupby('medium').filter(lambda x : len(x)>9)

In [67]:
data_df_clean

object,CHEBI:132943,CHEBI:133748,CHEBI:14321,CHEBI:15361,CHEBI:15428,CHEBI:15792,CHEBI:15963,CHEBI:16015,CHEBI:16133,CHEBI:16193,...,traits.pathways:thiocyanate_reduction,traits.pathways:thiosulfate_disproportionation,traits.pathways:thiosulfate_oxidation,traits.pathways:thiosulfate_oxidation_dark,traits.pathways:thiosulfate_reduction,traits.pathways:trithionate_oxidation,traits.pathways:uraninite_oxidation,traits.pathways:uranium_oxidation,traits.pathways:uranyl_reduction,medium
subject,,,,,,,,,,,,,,,,,,,,,
NCBITaxon:1001240,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,mediadive.medium:92
NCBITaxon:1004279,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,mediadive.medium:98
NCBITaxon:1004304,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,mediadive.medium:830
NCBITaxon:1004326,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,mediadive.medium:514
NCBITaxon:1005740,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,mediadive.medium:503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NCBITaxon:999545,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,mediadive.medium:514
NCBITaxon:999549,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,mediadive.medium:514
NCBITaxon:999550,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,mediadive.medium:514


In [68]:
# Splitting the data into features and target labels
X = data_df_clean.drop('medium', axis=1)#data_pairs_clean[['subject']]
y = data_df_clean['medium']

# Convert the input labels column to a categorical type if it isn't
#X['subject'] = X['subject'].astype('category')

# Convert categorical columns to integers
#X['subject'] = X['subject'].cat.codes

In [69]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Train the model
train_data = Pool(data=X_train, label=y_train, cat_features=[0])
test_data = Pool(data=X_test, label=y_test, cat_features=[0])


In [70]:
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, verbose=100)
model.fit(train_data)#, plot=True)

0:	learn: 3.5790903	total: 46.1ms	remaining: 46s
100:	learn: 2.2527962	total: 3.04s	remaining: 27s
200:	learn: 1.9924290	total: 6.01s	remaining: 23.9s
300:	learn: 1.8411058	total: 9s	remaining: 20.9s
400:	learn: 1.7401749	total: 12s	remaining: 17.9s
500:	learn: 1.6485124	total: 15s	remaining: 14.9s
600:	learn: 1.5819366	total: 17.9s	remaining: 11.9s
700:	learn: 1.5227685	total: 20.9s	remaining: 8.93s
800:	learn: 1.4760654	total: 24s	remaining: 5.96s
900:	learn: 1.4294069	total: 27s	remaining: 2.96s
999:	learn: 1.3942375	total: 29.9s	remaining: 0us


In [73]:
# Predict on test data
y_pred = model.predict(test_data)
y_pred_proba = model.predict_proba(test_data)[:,1]  # Probabilities for the positive class

# Print metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
#print("AUC-ROC:", roc_auc_score(y_test, y_pred_proba))

Accuracy: 0.3158995815899582

Classification Report:
                         precision    recall  f1-score   support

    mediadive.medium:1       0.12      0.15      0.13        26
  mediadive.medium:104       0.00      0.00      0.00        14
 mediadive.medium:104b       0.67      0.29      0.40         7
 mediadive.medium:104c       0.00      0.00      0.00         2
mediadive.medium:1076b       0.00      0.00      0.00         6
   mediadive.medium:11       0.35      0.33      0.34        21
  mediadive.medium:110       0.67      0.50      0.57         4
  mediadive.medium:119       0.80      1.00      0.89         4
  mediadive.medium:141       0.00      0.00      0.00         3
 mediadive.medium:141c       0.67      1.00      0.80         2
  mediadive.medium:194       0.00      0.00      0.00         1
 mediadive.medium:195c       0.00      0.00      0.00         2
   mediadive.medium:1a       0.00      0.00      0.00         7
  mediadive.medium:215       0.00      0.00      

/Users/marcin/Documents/VIMSS/ontology/KG-Hub/KG-Microbe/kg-microbe/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/marcin/Documents/VIMSS/ontology/KG-Hub/KG-Microbe/kg-microbe/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/marcin/Documents/VIMSS/ontology/KG-Hub/KG-Microbe/kg-microbe/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicte

In [72]:
# Predict on test data
y_pred = model.predict(train_data)
y_pred_proba = model.predict_proba(train_data)[:,1]  # Probabilities for the positive class

# Print metrics
print("Accuracy:", accuracy_score(y_train, y_pred))
print("\nClassification Report:\n", classification_report(y_train, y_pred))
#print("AUC-ROC:", roc_auc_score(y_test, y_pred_proba))

Accuracy: 0.6212676794133054

Classification Report:
                         precision    recall  f1-score   support

    mediadive.medium:1       0.58      0.54      0.56       153
  mediadive.medium:104       1.00      0.15      0.26        40
 mediadive.medium:104b       1.00      0.43      0.61        23
 mediadive.medium:104c       1.00      0.38      0.55         8
mediadive.medium:1076b       1.00      0.04      0.08        24
   mediadive.medium:11       0.59      0.76      0.66        83
  mediadive.medium:110       0.77      0.33      0.47        30
  mediadive.medium:119       0.83      0.86      0.84        22
  mediadive.medium:141       0.57      0.75      0.65        16
 mediadive.medium:141c       1.00      0.90      0.95        10
  mediadive.medium:194       1.00      0.31      0.47        13
 mediadive.medium:195c       0.86      0.50      0.63        12
   mediadive.medium:1a       0.94      0.62      0.75        24
  mediadive.medium:215       1.00      0.27      

In [60]:
len(y_test.unique())

43

In [19]:
import requests
from xml.etree import ElementTree
import pandas as pd

def get_taxon_genome_features(taxon_id):
    # Base URL for E-utilities
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
    
    # Use esearch to search for the taxon identifier in the taxonomy database
    search_url = f"{base_url}esearch.fcgi?db=taxonomy&term={taxon_id}&retmode=json"
    search_response = requests.get(search_url)
    search_data = search_response.json()
    
    # Obtain the Taxonomy ID from search response
    tax_id = search_data["esearchresult"]["idlist"][0]

    # Use efetch to get genome features using the taxonomy ID
    fetch_url = f"{base_url}efetch.fcgi?db=genome&id={tax_id}&rettype=fasta&retmode=xml"
    fetch_response = requests.get(fetch_url)
    genome_data = ElementTree.fromstring(fetch_response.content)
    
    # Parse the XML to extract genome features
    features = {}
    for feature in genome_data.findall('.//Feature'):
        feature_key = feature.find('Feature_key').text
        feature_value = feature.find('Feature_value').text
        if feature_key and feature_value:
            features[feature_key] = features.get(feature_key, []) + [feature_value]
    
    return features

def save_to_tsv(data, output_file):
    # Convert the dictionary to a pandas DataFrame
    df = pd.DataFrame.from_dict(data, orient='index')
    
    # Transpose the DataFrame so that taxon identifiers are row names
    df = df.transpose()
    
    # Save the DataFrame as TSV
    df.to_csv(output_file, sep='\t', header=True, index=True)

# Example Taxon ID
taxon_id = "NCBITaxon:100053"

# Strip 'NCBITaxon:' prefix to use with NCBI API
clean_taxon_id = taxon_id.replace("NCBITaxon:", "")

# Obtain genome features
genome_features = get_taxon_genome_features(clean_taxon_id)

# Specify the output file name
output_tsv = "genome_features.tsv"

# Save the genome features to a TSV file
save_to_tsv(genome_features, output_tsv)

print(f"Genome features saved to {output_tsv}")

IndexError: list index out of range

In [24]:
import requests

def get_uniprot_proteome_id(ncbi_taxon_id):
    # Remove the prefix 'NCBITaxon:' if present
    ncbi_taxon_id = ncbi_taxon_id.replace('NCBITaxon:', '')

    # UniProt search URL
    url = 'https://rest.uniprot.org/proteomes/search'
    params = {
        'query': f'taxonomy:{ncbi_taxon_id}',
        'format': 'list'
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        # Parse the response text for proteome IDs
        proteome_ids = response.text.strip().split('\n')
        if proteome_ids:
            return proteome_ids  # Return a list of found proteome IDs
        else:
            print(f"No proteome found for NCBI Taxon ID: {ncbi_taxon_id}")
            return None
    else:
        print(f"Failed to retrieve data from UniProt for NCBI Taxon ID: {ncbi_taxon_id}")
        return None

# Example usage
ncbi_taxon_id = 'NCBITaxon:100053'
clean_taxon_id = ncbi_taxon_id.replace("NCBITaxon:", "")
proteome_ids = get_uniprot_proteome_id(clean_taxon_id)

if proteome_ids:
    for proteome_id in proteome_ids:
        print(f"Found UniProtKB proteome identifier: {proteome_id}")


Failed to retrieve data from UniProt for NCBI Taxon ID: 100053


In [25]:
for taxon in data_pairs_clean.subject:
    taxon = taxon.replace("NCBITaxon:", "")
    proteome_ids = get_uniprot_proteome_id(taxon)

    if proteome_ids:
        for proteome_id in proteome_ids:
            print(f"Found UniProtKB proteome identifier: {proteome_id}")

Failed to retrieve data from UniProt for NCBI Taxon ID: 100053
Failed to retrieve data from UniProt for NCBI Taxon ID: 1000566
Failed to retrieve data from UniProt for NCBI Taxon ID: 1001240
Failed to retrieve data from UniProt for NCBI Taxon ID: 1002227
Failed to retrieve data from UniProt for NCBI Taxon ID: 1004279
Failed to retrieve data from UniProt for NCBI Taxon ID: 1004304
Failed to retrieve data from UniProt for NCBI Taxon ID: 1004316
Failed to retrieve data from UniProt for NCBI Taxon ID: 1004322
Failed to retrieve data from UniProt for NCBI Taxon ID: 1004326
Failed to retrieve data from UniProt for NCBI Taxon ID: 1004583
Failed to retrieve data from UniProt for NCBI Taxon ID: 1005
Failed to retrieve data from UniProt for NCBI Taxon ID: 1005740
Failed to retrieve data from UniProt for NCBI Taxon ID: 1005928
Failed to retrieve data from UniProt for NCBI Taxon ID: 1005944
Failed to retrieve data from UniProt for NCBI Taxon ID: 1005945
Failed to retrieve data from UniProt for NCB

KeyboardInterrupt: 

In [26]:
import requests
from xml.etree import ElementTree

def get_taxonomic_lineage(ncbi_taxon_id):
    # NCBI EFetch URL for taxonomy database
    efetch_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"

    # Parameters for fetching taxonomy information
    params = {
        "db": "taxonomy",
        "id": ncbi_taxon_id,
        "retmode": "xml"
    }

    # Make the request to NCBI EFetch
    response = requests.get(efetch_url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the XML response
        root = ElementTree.fromstring(response.content)
        # Find the lineage string
        lineage = root.find(".//Lineage")
        if lineage is not None:
            return lineage.text
        else:
            print(f"Lineage information not found for Taxon ID {ncbi_taxon_id}")
            return None
    else:
        print(f"Failed to retrieve data from NCBI for Taxon ID {ncbi_taxon_id}")
        return None

# Example usage:
ncbi_taxon_id = "9606"  # Homo sapiens Taxon ID
lineage = get_taxonomic_lineage(ncbi_taxon_id)

if lineage:
    print(f"Taxonomic lineage for NCBI Taxon ID {ncbi_taxon_id}: {lineage}")


Taxonomic lineage for NCBI Taxon ID 9606: cellular organisms; Eukaryota; Opisthokonta; Metazoa; Eumetazoa; Bilateria; Deuterostomia; Chordata; Craniata; Vertebrata; Gnathostomata; Teleostomi; Euteleostomi; Sarcopterygii; Dipnotetrapodomorpha; Tetrapoda; Amniota; Mammalia; Theria; Eutheria; Boreoeutheria; Euarchontoglires; Primates; Haplorrhini; Simiiformes; Catarrhini; Hominoidea; Hominidae; Homininae; Homo


In [28]:
lineages = []
counter =0
for taxon in data_pairs_clean.subject:
    taxon = taxon.replace("NCBITaxon:", "")
    print(f'index: {counter}\ttaxon:{taxon}')
    lineage = get_taxonomic_lineage(taxon)
    lineages.append(lineage)
    counter = counter +1 

index: 0	taxon:100053
index: 1	taxon:1000566
index: 2	taxon:1001240
index: 3	taxon:1002227
index: 4	taxon:1004279
index: 5	taxon:1004304
index: 6	taxon:1004316
index: 7	taxon:1004322
index: 8	taxon:1004326
index: 9	taxon:1004583
index: 10	taxon:1005
index: 11	taxon:1005740
index: 12	taxon:1005928
index: 13	taxon:1005944
index: 14	taxon:1005945
index: 15	taxon:1007092
index: 16	taxon:1007097
index: 17	taxon:1007099
index: 18	taxon:1007511
index: 19	taxon:1007676
index: 20	taxon:1009370
index: 21	taxon:1010610
index: 22	taxon:1010611
index: 23	taxon:101070
index: 24	taxon:101564
index: 25	taxon:1017270
index: 26	taxon:1017273
index: 27	taxon:1025
index: 28	taxon:1027633
index: 29	taxon:1028750
index: 30	taxon:1031540
index: 31	taxon:1031541
index: 32	taxon:1031542
index: 33	taxon:1032240
index: 34	taxon:103232
index: 35	taxon:1032476
index: 36	taxon:1032623
index: 37	taxon:1033734
index: 38	taxon:1033736
index: 39	taxon:1033739
index: 40	taxon:1033740
index: 41	taxon:1033806
index: 42	ta

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [ ]:
data_pairs_clean['lineage'] = lineages
file_name = 'taxa_media_lineages.tsv'
data_pairs_clean.to_csv(file_name, sep='\t', index=True)